In [124]:
import pandas as pd
import seaborn as sns
import numpy as np
import re

from textwrap import wrap
from utils import *
from matplotlib import pyplot as plt
from pymongo import MongoClient

In [157]:
acidentes_der = mongo_to_dataframe("acidentes_fatais_2008_2017")
acidentes_prf = mongo_to_dataframe("acidentes_rodovias_2010_2019") 

* Determinar causa de acidentes fatais em rodovias
    * Trabalho pode depender de perícia em certos casos

* Inputs:
    * condições climáticas
    * fase_dia, dia, mês
    * sentido_via (considerar)
    * uso_solo (considerar)
    * tipo_pista
    * tracado_pista
    * densidade da população
    * tipo_acidente
    * classificacao_acidente
    * pessoas
    * veiculos

### Preparação dos dados

In [134]:
def get_periodo(row, horario_column):
    if row[horario_column] == 'NAO_ESPECIFICADA':
        return 'NAO_ESPECIFICADA'
    elif wrap(row[horario_column], 2)[0] >= '06' and wrap(row[horario_column], 2)[0] < '12':
        return 'Amanhecer'
    elif wrap(row[horario_column] , 2)[0]>= '12' and wrap(row[horario_column] , 2)[0]< '18':
        return 'Pleno dia'
    elif wrap(row[horario_column] , 2)[0]>= '18' and wrap(row[horario_column] , 2)[0]< '19':
        return 'Anoitecer'
    elif wrap(row[horario_column] , 2)[0]>= '19' and wrap(row[horario_column] , 2)[0]<= '23':
        return 'Plena Noite'
    elif wrap(row[horario_column] , 2)[0]>= '00' and wrap(row[horario_column] , 2)[0]<= '06':
        return 'Madrugada'  
    

In [172]:
dias = {
    'segunda-feira': 'Segunda',
    'terça-feira': 'Terça',
    'quarta-feira': 'Quarta',
    'quinta-feira': 'Quinta',
    'sexta-feira': 'Sexta',
    'sábado': 'Sábado',
    'domingo': 'Domingo'
}

uso_solo = {
    'Sim': 'Urbano',
    'Não': 'Rural'
}

tipos = {
    'Derramamento de carga': 'Outros',
    'Derramamento de Carga': 'Outros',
    'Danos Eventuais': 'Outros',
    'Danos eventuais': 'Outros',
    'Atropelamento de Animal': 'Atropelamento de animal',
    'Colisão com objeto móvel': 'Colisão com objeto em movimento',
    'Colisão com objeto estático': 'Colisão com objeto fixo',
    'Saída de leito carroçável': 'Saída de Pista'
}

acidentes_prf = mongo_to_dataframe("acidentes_rodovias_2010_2019") 

columns_to_drop = ['data_inversa', 'horario', 'uf', 'ano', 'km', 'ignorados']

acidentes_prf.horario = acidentes_prf.horario.str.strip()
acidentes_prf['fase_dia'] = acidentes_prf.apply(lambda row: get_periodo(row, 'horario'), axis = 1)

acidentes_prf.dia_semana = acidentes_prf.dia_semana.replace(dias)

acidentes_prf.uso_solo = acidentes_prf.uso_solo.str.strip()
acidentes_prf.uso_solo = acidentes_prf.uso_solo.replace(dias)

acidentes_prf.horario = pd.to_datetime(acidentes_prf.horario)
acidentes_prf['mes'] = acidentes_prf.horario.dt.month

acidentes_prf.condicao_metereologica = acidentes_prf.condicao_metereologica.str.strip()
acidentes_prf.condicao_metereologica = acidentes_prf.condicao_metereologica.replace({'Céu Claro': 'Ceu Claro'})

acidentes_prf.tipo_pista = acidentes_prf.tipo_pista.str.strip()
acidentes_prf.tracado_via = acidentes_prf.tracado_via.str.strip()

acidentes_prf.tipo_acidente = acidentes_prf.tipo_acidente.str.strip()
acidentes_prf.tipo_acidente = acidentes_prf.tipo_acidente.replace(tipos)

acidentes_prf.classificacao_acidente = acidentes_prf.classificacao_acidente.str.strip()

#acidentes_prf.km = acidentes_prf.km.astype(str)
#acidentes_prf.km = acidentes_prf.km.replace({',':'.'})

#acidentes_prf.br = acidentes_prf.br.astype(str)
acidentes_prf.br = acidentes_prf.br.replace({',':'.'})

acidentes_prf = acidentes_prf.drop(columns_to_drop, axis = 1)
acidentes_prf = acidentes_prf.dropna()

acidentes_prf.br = acidentes_prf.br.astype(float).astype(int)

acidentes_prf.sentido_via = acidentes_prf.sentido_via.str.strip()
#acidentes_prf.km = acidentes_prf.km.astype(float).astype(int)

In [174]:
acidentes_prf

,dia_semana,br,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,feridos,veiculos,mes
0,Segunda,80,Falta de Atenção à Condução,Capotamento,Sem Vítimas,Madrugada,Crescente,Ceu Claro,Simples,Reta,Não,1,0,0,0,0,0,1,10
1,Segunda,70,Falta de Atenção à Condução,Colisão com objeto fixo,Com Vítimas Fatais,Madrugada,Crescente,Ceu Claro,Dupla,Curva,Não,2,1,1,0,0,1,1,10
2,Segunda,60,Falta de Atenção à Condução,Colisão com objeto fixo,Com Vítimas Fatais,Madrugada,Crescente,Ceu Claro,Dupla,Reta,Sim,4,2,0,2,0,2,1,10
3,Segunda,20,Condutor Dormindo,Colisão lateral,Com Vítimas Feridas,Amanhecer,Decrescente,Ceu Claro,Dupla,Reta,Sim,3,0,2,0,1,2,2,10
4,Segunda,70,Falta de Atenção à Condução,Colisão transversal,Com Vítimas Feridas,Pleno dia,Crescente,Nublado,Dupla,Reta,Sim,2,0,1,0,1,1,2,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11776,Sábado,70,Outras,Colisão lateral,Sem Vítimas,Amanhecer,Decrescente,Nublado,Dupla,Reta,Urbano,2,0,0,0,2,0,2,10
11777,Domingo,70,Falta de atenção,Colisão Transversal,Com Vítimas Feridas,Plena Noite,Decrescente,Nublado,Dupla,Cruzamento,Urbano,3,0,1,0,2,1,3,10
11778,Quinta,40,Defeito na via,Saída de Pista,Sem Vítimas,Pleno dia,Crescente,Ceu Claro,Simples,Reta,Urbano,1,0,0,0,1,0,1,10
11779,Quinta,20,Defeito mecânico em veículo,Capotamento,Sem Vítimas,Pleno dia,Crescente,Chuva,Dupla,Reta,Rural,1,0,0,0,1,0,1,10


In [145]:
acidentes_prf.km = acidentes_prf.km.astype(int)

AttributeError: 'DataFrame' object has no attribute 'km'

In [127]:
teste = '23:00:00'
teste2 = wrap(teste, 2)[0]
print(teste2)
if teste2 <= '23':
    print('true')
else:
    print('false')

23
true


In [169]:
acidentes_prf.sentido_via.value_counts()

Crescente               5528
Decrescente             5158
Crescente                579
Decrescente              511
Name: sentido_via, dtype: int64